In [38]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
import datetime
import pprint

In [2]:
pd.set_option('display.max_colwidth', -1)

### Get Forums and Subforums
Do not need to run this section if have the list of subforums

In [ ]:
def getForums(site_url):
    col=['forum','subforum','forum_url']
    top_df = pd.DataFrame(columns=col)
    r = re.get(site_url)
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    for element in soup.find_all('td', {"class": "alt1Active"}):
        forum_title_div = element.div
        forum_link = forum_title_div.a
        forum_url = site_url + forum_link['href']
        if('.com' in forum_link['href']):
            forum_url = forum_link['href']
        forum_title = forum_link.strong.string
        subforum = ''

        row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace

        for child in element.find_all('div', {"style" : "margin-top:6px"}):
            if child.find('strong') != None: #has a Sub-Forums section
                for subforum_link in child.find_all('a'):
                    subforum = subforum_link.contents[0]
                    forum_url = site_url + subforum_link['href']
                    if('.com' in subforum_link['href']):
                        forum_url = subforum_link['href']
                    row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
                    if(len(top_df)==0):
                        top_df = row
                    else:
                        top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return top_df

In [ ]:
site_url = 'https://forums.hardwarezone.com.sg'
hwz = getForums(site_url)
print(len(hwz))
hwz

In [ ]:
hwz.to_csv('hwz_subforums.csv')

### Get Threads

In [39]:
def getThreads(forum_url, keywords):
    lastForumPage = False
    forum_cols = ['thread', 'datetime','thread_url']
    forum_df = pd.DataFrame(columns=forum_cols)
    forum_page_url = forum_url
    
    while(not lastForumPage):
        r2 = re.get(forum_page_url)
        forum_page = r2.text
        forum_page_soup = BeautifulSoup(forum_page, 'html.parser')

        if (forum_page_soup.find('a', text='Next ›') == None):
            lastForumPage = True
        else:
            forum_page_url = site_url + forum_page_soup.find('a', text='Next ›')['href']
        
        forum_num = subforum_url.strip('/').split('-')
        forum_num = forum_num[-1]
        
        thread_body = forum_page_soup.find('tbody',{'id': f'threadbits_forum_{forum_num}'})
        
        for thread in thread_body:
            if len(thread) == 1: 
                continue
            try: 

                # Retrieve Date 
                date_block = thread.find_all('td', class_ = "alt2")[1].find('div', class_= "smallfont" ).contents
                date = date_block[0].strip()
                if date == 'Yesterday' : 
                    date = (datetime.date.today() - datetime.timedelta(1)).strftime('%Y-%m-%d')
                elif date == 'Today': 
                    date = datetime.date.today().strftime('%Y-%m-%d')
                    
                if '2020' not in date and '2021' not in date: 
                    if 'class="hwz-sticky"' not in str(thread):
                        return forum_df   # Return forum_df with threads from 2020 onwards
                    else:
                        continue

                # Retrieve Thread Title
                title = thread.find('a', {"id": lambda x: x and 'thread_title_' in x}).contents[0]
                for k in keywords: 
                    if k in title.lower():

                        print(title)

                        # Retrieve Time
                        time = thread.find('span', class_ = 'time').text

                        # Retrieve Date and Time
                        date_time = date + ' ' + time

                        # Retrieve Thread Url           
                        url = site_url + thread.find('a', {"id": lambda x: x and 'thread_title_' in x})['href']        

                        row = pd.DataFrame([[title, date_time, url]], columns=forum_cols)
                        if(len(forum_df)==0):
                            forum_df = row
                        else:
                            forum_df = forum_df.append(row, ignore_index=True) #df.append doesn't work inplace

            except:
                continue
                
    forum_df.drop_duplicates()
                
    return forum_df

#### List of keywords
As of 26/02/2021

In [32]:
kw_tracetogether = ['tracetogether', 'trace together', 'contact tracing', 'contacttracing', 'tokens', 'breach of privacy', 
                    'criminal data', 'bluetrace', 'opentrace', 'nric', 'ministry of heath', 'govtech'
                    'government technology agency', 'smart nation', 'vivian balakrishnan', 'bluetooth', 'lawrence wong', 
                    "singapore's people party", 'criminal procedure code', 'surveillance', 'privacy', 'eugene tan']

kw_safeentry = ['safeentry', 'safe entry']

kw_vaccination = ['vaccine', 'vaccination', 'pfizer', 'moderna', 'vaccine side effects', 'sinovac']

kw_foreign_workers = ['foreign worker', 'foreign worker dorm', 'foreign worker dormitories', 'dorm', 'dormitories', 
                      'swab testing facilities', 'dormitory cluster', 'mom', 'ministry of manpower',
                      'community care facility', 'community care facilities', 'josephine teo', 'migrant workers', 
                      'makeshift dormitories']

kw_social_distancing = ['social distancing', 'safe distancing', 'group size limit', '1m apart', '2m apart', 'cny', 
                        'chinese new year', 'safe distancing ambassador', 'social distancing ambassador', 
                        'physical contact', 'face-to-face']

kw_circuit_breaker = ['circuit breaker', 'cb', 'panic buying', 'hoarding', 'quarantine', 'lockdown', 'travel bubble',
                      'travel restrictions', 'work from home', 'wfh', 'telecommuting', 'remote working', 'home based learning', 
                      'hbl', 'stay home order', "mcdonald's", 'haircut', 'hair salon', 'hair dressers', 'barber', 'gathering', 
                      'mass gathering', 'places of worship', 'place of worship', 'marriage', 'empty streets', 'airplane crew', 
                      'stay home', 'airline crew', 'sia', 'reopening', 'stewardess', 'staycation', 'house visit', 'gather', 
                      'extension']

kw_economic_measures = ['government spending', 'budget', 'heng swee keat', 'economy', 'jobs support scheme', 'stimulus package', 
                        'support fund', 'stablisation and support package', 'assurance package', 'resilience budget', 'support package', 
                        'business support', 'co-funding', 'food service', 'solidarity budget', 'fortitude budget', 'wage subsidies', 
                        'jobs and skills package', 'singaporedediscovers', 'financial support', 'payouts', 'support grant', 
                        'temporary relief refund']

kw_general = ['phase 1', 'phase 2', 'phase 3', 'coronavirus', 'corona', 'covid', 'covid19', 'pandemic', 'healthcare', 'hospital', 
              'frontline workers', 'surgical face masks', 'face mask', 'masks', 'mask-wearing', 'hand sanitizer', 'clusters',
              'doscorn', 'stay home notice', 'shn', 'covid taskforce', 'swab test', 'Gan Kim Yong', 'local case',
              'sgunited', 'essential workers', 'non-essential workers', 'essential business', 'non-essential business', 
              'mental health']

#### Getting threads based on policy/keywords

In [40]:
'''Create a folder named "Threads" in the current directory'''

def threads_tracetogether(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_tracetogether_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_safeentry(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_safeentry_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_vaccination(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_vaccination_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_foreign_workers(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_foreign_workers_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_social_distancing(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_social_distancing_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_circuit_breaker(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_circuit_breaker_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_economic_measures(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_economic_measures_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

def threads_general(subforum, subforum_url, keywords): 
    threads = getThreads(subforum_url, keywords)
    filepath = os.path.join('Threads', f'{subforum}_general_threads.csv')
    threads.to_csv(filepath, encoding = 'utf-8', index = False)
    return threads.info()

In [ ]:
%%time
site_url = 'https://forums.hardwarezone.com.sg'

# manually key in subforum_url & subforum
subforum_url = 'https://forums.hardwarezone.com.sg/current-affairs-lounge-17/'
subforum = 'current-affairs-lounge'

# Change the directory
os.chdir(r"C:/Users/Gi Han/Documents/SMT203 Computational Social Science/Hardwarezone")

threads_tracetogether(subforum, subforum_url, kw_tracetogether)
threads_safeentry(subforum, subforum_url, kw_safeentry)
threads_vaccination(subforum, subforum_url, kw_vaccination)
threads_foreign_workers(subforum, subforum_url, kw_foreign_workers)
# threads_social_distancing(subforum, subforum_url, kw_social_distancing)
# threads_circuit_breaker(subforum, subforum_url, kw_circuit_breaker)
# threads_economic_measures(subforum, subforum_url, kw_economic_measures)
# threads_general(subforum, subforum_url, kw_general)

Communist China Exploits Gaps in US MilitaryRebeccah Heinrichs
Picture privacy.
Somebody stole my NRIC number???
[158th]Lawrence Wong:Doctors/experts requested MOM to advise employers not to send FW for testing!
COVID-19 contact tracing absolutely essential; wearable TraceTogether tokens to be rolled out in J
COVID-19 contact tracing absolutely essential; wearable TraceTogether tokens to be rolled out in J
COVID-19 contact tracing absolutely essential; wearable TraceTogether tokens to be rolled out in J
[BREAK] Contact tracing devices to be rolled out 'soon', distributed to everyone in Singapore
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
thread        8 non-null object
datetime      8 non-null object
thread_url    8 non-null object
dtypes: object(3)
memory usage: 320.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
thread        0 non-null object
datetime      0 non-null objec

### Get Posts - all pages

In [16]:
def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['post_id','timestamp', 'post_text']
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url
    
    count = 0
    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', text='Next ›') == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', text='Next ›')['href']

        thread_page_posts = thread_page_soup.find('div', {'id': 'posts'})
                
        try: 
            for post in thread_page_posts.find_all('div', {'class': 'post-wrapper'}):
                
                # Retrieve Date & Time
                datetime_raw = post.find('a', {'name': lambda x: x and x.find('post') == 0}).nextSibling.strip()
                date = datetime_raw.split(',')[0]
                
                if date == 'Yesterday' : 
                    date = (datetime.date.today() - datetime.timedelta(1)).strftime('%Y-%m-%d')
                elif date == 'Today': 
                    date = datetime.date.today().strftime('%Y-%m-%d')
                    
                time = datetime_raw.split(',')[1]
                date_time = date + ' ' + time
                
                # Retrieve Post Text
                post_text = ""
                try:
                    post_text = post.find('div', {'class': 'post_message'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass
                
                # Retrieve Post ID
                post_id = int(post.find('a', {'id': lambda x: x and 'postcount' in x, 'target': 'new'})['id'].lstrip('postcount'))
                
                row = pd.DataFrame([[ post_id, date_time, post_text]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[np.nan, np.nan, ""]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        
    thread_df['post_text'] = thread_df['post_text'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))
    
    thread_df.drop_duplicates()

    return thread_df

#### Combine posts with threads

In [4]:
# Returns posts for each thread in specified subforum and policy 

def post_threads(df):
    new_df = pd.DataFrame(columns = ['thread', 'datetime','thread_url', 'post_id','post_timestamp', 'post_text'])
    df_col = new_df.columns
    
    for index in df.index:
        thread_url = df['thread_url'][index]
        posts = getPosts(thread_url)
        for p_index in posts.index:
            row = pd.DataFrame([[df['thread'][index], df['datetime'][index], df['thread_url'][index], 
                                posts['post_id'][p_index], posts['timestamp'][p_index], posts['post_text'][p_index]]], 
                               columns = df_col)
            new_df = new_df.append(row, ignore_index = True, sort = False)
    return new_df

In [11]:
cwd = os.path.abspath('.')

# Change the directory to the respective directory
os.chdir(r"./Threads/..")

files = os.listdir(cwd)
files

['eat-drink-man-woman_vaccination_threads.csv',
 'Not Done',
 'Post Retrieved',
 'Posts']

In [10]:
'''Create a folder named "Posts" in the current directory'''

site_url = 'https://forums.hardwarezone.com.sg'

for file in files:
    if file.endswith('.csv'):
        df = pd.read_csv(f'{file}')
        new_df = post_threads(df)
        filename = f'{file}'[:-11] + 'posts'
        filepath = os.path.join('Posts', filename + '.csv')
        new_df.to_csv(filepath, index = False)
        print(f'Complete: {file}')

Complete: eat-drink-man-woman_tracetogether_threads.csv
Complete: eat-drink-man-woman_vaccination_threads.csv
